# DataDome Assignment

In [ ]:
'''
Identify good bot traffic, i.e. non malicious bots like such as Google Bot
'''
good_bot_selectors = [
    # Google Ads Bot (AdsBot-Google)
    # Arquivo web crawler (Heritrix)
    # Twitter Bot
    # WordPress related bots
    # LinkedIn Bot
    # HubSpot Ads Link Fetcher
    # Feedly Bot
    # Capterra Bot
    # Asana - Project management tool
    # Apache-HttpClient - widely used Java library for making HTTP requests
    # AliyunSecBot - Associated with Alibaba Cloud's security scanning service
    df['UserAgent'].str.contains(r'AdsBot-Google \(+', na=False),
    df['UserAgent'].str.contains(r'Arquivo-web-crawler \(compatible', na=False),
    df['UserAgent'].str.contains(r'Twitterbot\/1\.', na=False),
    df['UserAgent'].str.contains(r'WordPress\/6\.', na=False),
    df['UserAgent'].str.contains(r'LinkedInBot\/1.0', na=False),
    df['UserAgent'].str.contains(r'HubSpot Ads Link Fetcher\/1\.', na=False),
    df['UserAgent'].str.contains(r'Feedly\/1\.', na=False),
    df['UserAgent'].str.contains(r'CapterraBot', na=False),
    df['UserAgent'].str.contains(r'Asana\/1\.4\.0 WebsiteMetadataRetriever', na=False),
    df['UserAgent'].str.contains(r'Apache-HttpClient\/4\.', na=False),
    df['UserAgent'].str.contains(r'AliyunSecBot\/Nutch\-', na=False)
]

In [ ]:
'''
Identify bad bot traffic. We consider as malicious, all requests from non-identified bots (bots that
don’t explicitly state their identity in the user-agent), as well as fake good bots, i.e. bots that pretend
to be a good bot in their user-agent (fake google bot from IPs that do not belong to Google). You
should use different detection signals and different type of approaches (signatures vs behavioral
analysis)
'''
bad_bot_selectors = [
    #bxss\.me - Blind Cross-Site Scripting (Blind XSS) Bots
    #jndi:ldap:// - Outbound LDAP request - highly suspicious/malicious
    #oast\.me - Commonly associated with Out-of-Band Application Security Testing (OAST) tools
    #nslookup - Malicious DNS Lookup commands
    #\$\{sys:os\.arch\} - System property lookup, highly suspicious for legitimate traffic
    #\(select\(0\)from\(select\(sleep\( - Time-based SQL injection attacks to delay the server's response and confirm the presence of a vulnerability
    #&echo|\|echo - Command Injection (echo)
    #@@o9MoK - Suspicious String
    #useragent.m413k3936p0j44928gn68xr1otn7271 - Suspicious String
    #'../../' - This pattern is commonly associated with path traversal attempts
    #'\containers/json' - detects attempts to access Docker APIs
    #allow_url_include|auto_prepend_file - detect attempts to manipulate PHP settings 
    #fingerprintReferrer- malicious IPs in fingerprintReferrer can be used to detect malicious traffic
    #fingerprintClientIp - malicious IPs in fingerprintReferrer can be used to detect malicious traffic
    df['UserAgent'].str.contains(r'bxss\.me', na=False),
    df['UserAgent'].str.contains(r'\$\{jndi:ldap://', na=False),
    df['UserAgent'].str.contains(r'oast\.me', na=False),
    df['UserAgent'].str.contains(r'nslookup', na=False),
    df['UserAgent'].str.contains(r'\$\{sys:os\.arch\}', na=False),
    df['UserAgent'].str.contains(r'\(select\(0\)from\(select\(sleep\(', na=False),
    df['UserAgent'].str.contains(r'&echo|\|echo', na=False),
    df['UserAgent'].str.contains(r'@@o9MoK', na=False),
    df['UserAgent'].str.contains(r'useragent.m413k3936p0j44928gn68xr1otn7271', na=False),
    df['fingerprintRequestUrl'].str.contains(r'\/\.\.\/\.\.\/', na=False),
    df['fingerprintRequestUrl'].str.contains(r'\/containers\/json', na=False),
    df['fingerprintRequestUrl'].str.contains(r'allow_url_include|auto_prepend_file', na=False, case=False),
    df['fingerprintReferer'].str.contains(r'\/\/38.43.93.42', na=False, case=False),
    df['fingerprintClientIp'].str.contains(r'\/\/206.123.130.4', na=False, case=False)   
]

In [ ]:
'''
Identify human traffic
'''
humanTraffic = [
    # Popular Browsers/OS alogn with the version number
    # fingerprintRequestJsWebDriver - True for automation software, strong bot indication
    # fingerprintRequestJsWebGlRend - Identifies WebGL rendering capabilities of a browser. GPU names give confidence about real devices and not headless browsers
    # fingerprintRequestJsHardwareConcu - Captures the hardware concurrency. Not unique on its own but when used with other indicators that can make a good signature/signal
    df['UserAgent'].str.contains(r'Mozilla\/5\.0 \((Windows NT|Linux|Macintosh|SS|CentOS|Debian|Fedora|iPad|iPhone|Knoppix|X11)', na=False, case=False),
    df['UserAgent'].str.contains(r'Opera\/(\d+\.\d+)', na=False, case=False),
    df['UserAgent'].str.contains(r'Microsoft Office\/16\.0 \(', na=False, case=False),
    df['fingerprintRequestJsWebDriver'].astype(str).str.fullmatch('FALSE', case=False, na=False),
    (df['fingerprintRequestJsWebDriver'].astype(str).str.fullmatch('FALSE', case=False, na=False)) &
    (df['fingerprintRequestJsWebGlRend'].str.contains('Adreno|Nvidia|ANGLE|Apple|Google|Intel|Mali|Radeon', case=False, na=False)) &
    (df['fingerprintRequestJsHardwareConcu'].astype(str).str.match(r'\d+', na=False))
]

## Discuss your approaches/hypotheses when identifying both good and bad bots

Q. Is it based on behavior, HTTP headers, JS information, hybrid information? Also, if you don't use
some of these approaches, please explain your choice.

For the purpose of this assignment I considered a few indicators 
- fingerprintClientIp - Malicious IPs or IPs that have detection on popular security portals like VirusTotal can be used as an indicator. Since this IP signifies the IP of the client, a malicious IP can be used as a good indicator for identification
- Positives
    - Fast detection, easy to automate
Negatives
    - Its possible that an IP does not have detection when analysis was being done
    - If an IP is flagged because of a False Posivie by a different vendor we might get a False Positive if we use this detection as is

- fingerprintUserAgent - This field contains useragent of the client. In the given assignment strings present in this field can be used as a good indicator to identify malicious a well as non-malicious traffic
- Positives
    - Fast detection, easy to automate
    - Possible to build a respository of signals/rules for detection
- Negatives
    - Spoofing the content in this field can throw off detection

- fingerprintReferer - This field indicates the referral URL that can be checked for legitimacy. This can indicate where the request came from - a legitimate company, a service provider - which can give hints about the origins and purpose of the traffic
- Positives
    - Fast detection, easy to automate
    - Large number of requests coming from a service provider, home network along with other indicators can indicate suspicious activity (paid bot service usage etc)
- Negatives
    - Spoofing can be used here, use of anonymizers can lead to dead end as well
    - Rapid IP switching can also lead to dead end
- fingerprintRequestJsWebGlRend

The technique of combining multiple signals is useful in creating robust signatures. In thsi case I used a combination signature to identify good traffic (mostly human traffic). This included using - 
    1. fingerprintRequestJsWebDriver - True indicates use of automation tools
    2. fingerprintRequestJsWebGlRend - WebGL rendering capabilities of a browser, indicating use of actual browsers
    3. fingerprintRequestJsHardwareConcu - Hardware concurrency, also indicating normal traffic

Lastly I ran a script that checks Virustotal for any detections against a URL. I chose a bunch of IPs from the list and obtained few IPs that have detection. This can be used to further analyze the traffic as only the IP might not be a good indicator as external detections can be False Positives as well.
IP can be a good indicator as the below 2 entries seem legitimate considering other headers, but malicious IP address gives an indication that more analysis might be warranted

Dec 12, 2024 @ 13:57:29.770	34.92.43.64	http	text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8	datadome.co	Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36 (StatusCake)		/	gzip, compress	GOOGLE-CLOUD-PLATFORM	HK	en-US,en;q=0.8														

Dec 12, 2024 @ 17:00:31.840	34.141.240.181	http	text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8	datadome.co	Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:109.0) Gecko/20100101 Firefox/115.0	https://reddit.com	/fr/equipe-datadome/terminer-en-force-demarrer-rapidement-lequipe-de-vente-americaine-de-datadome/	gzip, br, zstd, deflate	GOOGLE-CLOUD-PLATFORM	NL															



## Summarize the main strengths and weaknesses of your approaches. Is there any risk of false positives?

I have highlighted positives and negatives for most of the indicators that I have used above. In most cases the risk of spoofing poses a threat to cause false positive/negative. We might identify malicious traffic as legitimate due to a rule that was created for a specific component - user agent for example. One way to overcome this is to use multiple signatures in combination rather than a single signature.

One of the main strengths in static signatures is that it can detect a big portion of malicious traffic as most of the times perpetrators create tools/services that target broad audience, thereby are created in a quick-and-dirty fashion. This approach lacks mechanisms to counteract signature based security systems. Challenge is to identify the 10-20% of attaks which are designed with a lot of attentino and which try to circumvent security mechanisms.

## What would you focus on improving next?

Given more time I would like to create more combination-based signatures that take multiple criterias into consideration. These might not catch as many entries as generic signatures but these would be more robust and less prone to false positives


## Imagine a customer wants to know more about bot traffic. Create some relevant visualizations (2-3 are enough) to help her/him understand what bots are doing on her/his website and the associated threats. 

I would take the liberty to provide details that are out of the given test material. 

I would begin by making the customer think about the possibility of losing customers because of bots as that affects their business directly. I would take uses cases to further hit my point. 

For this scenario I will consider that the customer owns a business that sells sneakers

## Overall, what can you say about the traffic that you analyzed?
THe traffic analyzed was a mix of malicious and non-malicious traffic
- Malicious traffic included some attempts to exploit the systems using known methods. These can be identified using signals/signatures and this can be improved as we obtain more data
- Non-malicious traffic included good bot traffic mainly related to crawling and indexing
- There was a lot of human traffic as well which could be identified by few key indicators
- Lastly there was human traffic which appeared malicious based onthe IP, these would require further invetigation

Overall it gave a good glimpse of regular internet traffic which includes a lof of good as well as bad network activity. It also gave a good high level view of what DataDome researchers have to comb through on a regula basis.
